# A simple parser for LTspice files

**Scott Prahl**

May 2022

This notebook treats files with op-amps LTspice files back to a simple netlist.

In [ ]:
import matplotlib.pyplot as plt
import lcapy
import ltparser

#make matplotlib hires
%config InlineBackend.figure_format='retina'

In [ ]:
lt = ltparser.LTspice()
lt.read('../tests/examples/inverting-opamp.asc')
print(lt.contents)

In [ ]:
lt.parse()
lt.make_netlist()
lt.print_nodes()

In [ ]:
lt.plot_nodes()


In [ ]:
lt = ltparser.LTspice()
lt.read('../tests/examples/audio-crossover2.asc')
lt.make_netlist()
lt.plot_nodes()
print(lt.netlist)

In [ ]:
cct=lt.circuit()
cct.draw(scale=0.5)

In [ ]:
import imageio
im = imageio.imread('../tests/examples/audio-crossover.png')
plt.figure(figsize=(14,6))
plt.imshow(im)
plt.show()

In [ ]:
print(lt.netlist)

In [ ]:
lt.make_graph()

In [ ]:
cct=lt.circuit()
cct.draw(scale=0.5)